In [11]:
from quantumvoting import Voter, Authenticator, Tallier, Blockchain
import time

CHOICES = ["YES", "NO", "ABSTAIN"]

VOTER_CHOICES = {
    "Aya": "YES",
    "Bassem": "NO",
    "Charlie": "ABSTAIN",
    "Dana": "NO",
    "Ella": "NO",
}

QKD_KEY_LENGTH = 32

In [12]:
voters = [Voter(name, qkd_key_length=QKD_KEY_LENGTH) for name in VOTER_CHOICES.keys()]
voter_dict = {voter.id: voter.public_key for voter in voters}

auth = Authenticator("AuthNode", voter_dict, qkd_key_length=QKD_KEY_LENGTH)

tally1 = Tallier("TallyNode1", qkd_key_length=QKD_KEY_LENGTH)
tally2 = Tallier("TallyNode2", qkd_key_length=QKD_KEY_LENGTH)
tally3 = Tallier("TallyNode3", qkd_key_length=QKD_KEY_LENGTH)
talliers = [tally1, tally2, tally3]

blockchain = Blockchain(talliers)

In [13]:
for voter in voters:
    voter.establish_qkd_channel(auth)
    for tally in talliers:
        voter.establish_qkd_channel(tally)

for tally in talliers:
    auth.establish_qkd_channel(tally)

----------------------------------------
--- QKD Session: Aya ↔ AuthNode ---
----------------------------------------
Negotiating QKD key... ################# Done.

Total qubits sent: 52
Total Eve attacks: 7

--- Eve Attack Log ---
Qubit 9: Eve basis=Z, Sender basis=X ➔ Flipped
Qubit 16: Eve basis=Z, Sender basis=X ➔ Flipped
Qubit 17: Eve basis=X, Sender basis=Z ➔ Flipped
Qubit 24: Eve basis=X, Sender basis=Z ➔ Flipped
Qubit 28: Eve basis=Z, Sender basis=X ➔ Flipped
Qubit 32: Eve basis=X, Sender basis=Z ➔ Flipped
Qubit 48: Eve basis=Z, Sender basis=X ➔ Flipped
----------------------

Aya basis commitment (hash): b9e1a86fb7a220f9230e230e2cbdcb996ca0912008d77ac00bb73431b8156edd
AuthNode basis commitment (hash): 96bb1b705c2595308057ed756da750cf60dd6b882599158bc95978da1efa2311
Basis commitments verified successfully.

Final sifted key before privacy amplification: [0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0]

Sample error rate: 0.00% (al

In [14]:
for voter in voters:
    voter.sign_identity()
    voter_choice = VOTER_CHOICES[voter.name]
    vote_index = CHOICES.index(voter_choice)

    vote_payload = voter.send_vote(vote_index, auth, talliers[0])

    signature_hex, forwarded_vote = auth.forward_vote(vote_payload, talliers[0], voter)

    for tally in talliers:
        tally_vote = tally.receive_vote(forwarded_vote, auth)
        tally.add_vote(signature_hex, tally_vote)

In [15]:
first_tallier = talliers[0]
votes_snapshot = first_tallier.received_votes.copy()
standard_timestamp = time.time()

proposed_blocks = []
for tally in talliers:
    block = blockchain.propose_block(
        votes_snapshot,
        tally,
        fixed_creator="TallyConsensus",
        fixed_timestamp=standard_timestamp
    )
    proposed_blocks.append(block)

if blockchain.byzantine_agreement(proposed_blocks):
    blockchain.display_chain()
    blockchain.save_json("blockchain.json")
    blockchain.simple_majority()

Consensus reached. Block 1 added.

Blockchain:
Block 0 [Creator: System]
  Timestamp: Sun Apr 27 15:56:11 2025
  Previous Hash: 0
  Hash: fdffa441663fbae798609d9fc1679f3a01ac2d0e3bdba4fd40f4fac9ad2d55a0
  Data: Genesis Block
----------------------------------------
Block 1 [Creator: TallyConsensus]
  Timestamp: Sun Apr 27 16:01:26 2025
  Previous Hash: fdffa441663fbae798609d9fc1679f3a01ac2d0e3bdba4fd40f4fac9ad2d55a0
  Hash: bb79b8403538505daef5c049226c261ba197b56fab8d2f99f3d31075550f1b47
  Data: {'b0c9dd0df23a004a7dc9985686c230f6942c391d5a97d4695887e19cdda2973bb498fd091fe64d0c630bfd0c30df56acc18fcc18fe1d1431b6ae12aff0ff03d2ef551e945320c9385a605c2aeeb13842334521d51d9dcc9809b567cbe1ca7797aaf18d3cf829e933b23b7fea14d1123d809d1001750c726b5a0589d24d2bd57d': '000', '88bd88b6e128b5d17c08a7673bc69a4b3dccb015e29f7f89e47f6730f1866789723bf2095fca3027021fef9aef061e6018da3c5b849ad70f66e89b47ef09419993314b5c89e7118fce825b858902e8c70d8d55a2a502f6210a0e2a7e06a75ab8ea1c699f33d9ccc6228b6c08ac7aba88bb522b